In [1]:
# is this needed ?
# import os
# os.environ["HF_DATASETS_CACHE"] = "/workspace/.cache/huggingface/datasets"
# os.environ["TRANSFORMERS_CACHE"] = "/workspace/.cache/huggingface/models"

import os

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AdamW
from huggingface_hub import HfApi

import util

log_board = util.LogBoard("log_dir", 6005)
log_board.launch()

log_every = 10
step_every = 4
save_every = 500
bin = "path/to/bin"
tokenizer_id = "meta-llama/Meta-Llama-3-8B"
model_id = "meta-llama/Meta-Llama-3-8B"

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

E0424 23:17:57.126233 140677271708736 program.py:300] TensorBoard could not bind to port 6005, it was already in use
ERROR: TensorBoard could not bind to port 6005, it was already in use


In [2]:

# model & tokenizer

model = util.model.build_tuneable_hf_model(model_id, quantized=True, device_map='auto')
train_tokenizer = AutoTokenizer.from_pretrained(tokenizer_id, trust_remote_code=True)
train_tokenizer.pad_token = train_tokenizer.eos_token
train_tokenizer.padding_side = "left"
train_tokenizer.add_bos_token = False
train_tokenizer.add_eos_token = False
train_tokenizer.__class__ = util.TrainingTokenizer
train_tokenizer.max_seq = 512

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

AttributeError: 'LlamaDecoderLayer' object has no attribute 'attn'

In [ ]:

# dataset

data_root = '../data'

instruct_tune_dataset = load_dataset( 'json', data_files={
    'train': f'{data_root}/train.jsonl', 
    'test': f'{data_root}/test.jsonl'
}).map(lambda x: {
    "conversations": util.convo.preprocess_sample(x['conversations'])
})

In [ ]:

# finetune

logger = log_board.get_logger("train")


optimizer = AdamW(model.parameters(), lr=5e-5)
train_loader = DataLoader(instruct_tune_dataset['train'], batch_size=1, shuffle=True)
def collator(batch):
    return train_tokenizer.pad([*map(lambda x: train_tokenizer(x), batch['conversations'])], return_tensors="pt", padding="max_length", max_length=max_seq_length)


i = 0
model.train()
for epoch in range(1000):  # number of epochs
    for batch_idx, batch in enumerate(train_loader):
        i += 1

        # prepare inputs
        inputs = collator(batch)
        labels = inputs["input_ids"].clone()
        padding = torch.full((labels.shape[0], 1), -100, dtype=torch.long, device=labels.device)  # Pad with -100 on the right
        labels = torch.cat([labels, padding], dim=1)[:, 1:]

        # forward backward
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()

        if i % step_every == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % log_every == 0:
            logger.add_scalar("Loss/train", loss.item(), epoch * len(train_loader) + batch_idx)

        if i % save_every == 0:
            os.makedirs(bin, exist_ok=True)
            util.models.save_model(model, save_path=f"{bin}/model-tuned.pth")

In [ ]:

# upload to huggingface
# REMEMBER to actually move model data into folder first

api = HfApi()
api.upload_folder(
    repo_id = "host_name/model_name",
    folder_path = "/path/to/model/and/parts"
)